<a href="https://colab.research.google.com/github/silvererudite/ML_algos_onSomeDatasets/blob/master/GSR_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')

!pip install neurokit2
!pip install pysiology

#some libraries are not used
import csv
import pickle
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from statistics import mean
import csv
import pickle
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from statistics import mean
from scipy.signal import welch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import neurokit2 as nk
import pysiology as pl

labels = pd.DataFrame()
for m in range(1,33):
    fl = f'/content/drive/My Drive/Deap Dataset/Copy of s{m}.dat'
    with open(fl, 'rb') as fl:content = pickle.load(fl, encoding='latin1')
    y=content["labels"]
    y=pd.DataFrame(y, columns=["valence", "arousal","dominance","likings"])
    y=y.iloc[:,0:4]
    labels=pd.concat([labels, y], ignore_index=True)

labels['Valence'] = labels['valence'].apply(lambda x: "0" if x <= 5 else "1")
labels['Arousal'] = labels['arousal'].apply(lambda x: "0" if x <= 5 else "1")
labels['Dominance'] = labels['dominance'].apply(lambda x: "0" if x <= 5 else "1")
labels['Likings'] = labels['likings'].apply(lambda x: "0" if x <= 5 else "1")

labels=labels.iloc[:,4:8]
labels.shape

labels.head


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<bound method NDFrame.head of      Valence Arousal Dominance Likings
0          1       1         1       1
1          1       1         1       1
2          1       1         1       1
3          0       1         1       1
4          1       0         1       1
...      ...     ...       ...     ...
1275       0       1         1       0
1276       0       1         1       0
1277       0       1         1       0
1278       0       1         0       0
1279       1       0         1       0

[1280 rows x 4 columns]>

In [35]:
features = pd.DataFrame()
channel_no=[36] #selecting channels
for m in range(1,33):

  print(m,end=",")

  fl = f'/content/drive/My Drive/Deap Dataset/Copy of s{m}.dat'

  with open(fl, 'rb') as fl: 
    content = pickle.load(fl, encoding='latin1')

  for i in range(0,40): #videos
    
    x=content["data"][i]
    feature_list=[]
    df2=pd.DataFrame()

    for j in channel_no: #channels

      gsr= x[j]

      x = pl.electrodermalactivity.analyzeGSR(gsr.tolist(), samplerate=128, preprocessing=False)

      check_list=['peak', 'riseTime', 'latency', 'amplitude', 'halfAmplitude', 'halfAmplitudeIndex', 'halfAmplitudeIndexPre', 'EDAatApex', 'decayTime', 'SCRWitdth']

      for key, val in x.items():

        if len(check_list) == len(val.keys()):

          feature_list.append(x[key]['peak']['peakEnd'])
          feature_list.append(x[key]['peakMax'] ['peakMax'])
          feature_list.append(x[key]['peakStart']['peakStart'])

          feature_list.append(x[key]['EDAatApex'])
          feature_list.append(x[key]['SCRWitdth'])
          feature_list.append(x[key]['halfAmplitudeIndexPre'])
          feature_list.append(x[key]['decayTime'])

          feature_list.append(x[key]['amplitude'])
          feature_list.append(x[key]['halfAmplitude'])
          feature_list.append(x[key]['halfAmplitudeIndex'])
          feature_list.append(x[key]['latency'])
          feature_list.append(x[key]['riseTime'])
          break


    d_f1 = pd.DataFrame(feature_list)
    df=d_f1.to_numpy()
    data=np.reshape(df,(1,-1))
    df2=pd.concat([df2,pd.DataFrame(data=data)],ignore_index=True)

    features=pd.concat([features, df2],  ignore_index=True)

#print(feature_list)
Features= pd.concat([features, labels], axis=1, sort=False)
Features = Features.dropna(how='any',axis=0)

val=Features.iloc[:,:]
#val.rename(columns={ val.columns[:,0:11]: check_list }, inplace = True)
val.to_csv(r'feature_GSR.csv', index = False, header=True)

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,

In [36]:
# features = pd.DataFrame()

# channel_no=[38] #selecting channels

# for m in range(1,33):

#   print(m,end=",")

#   fl = f'/content/drive/My Drive/Deap Dataset/Copy of s{m}.dat'

#   with open(fl, 'rb') as fl: content = pickle.load(fl, encoding='latin1')

#   for i in range(0,40): #videos
    
#     x=content["data"][i]

#     df2=pd.DataFrame()

#     for j in channel_no: #channels

#       ppg= x[j]  
          
#       ppg_cleaned = nk.ppg_clean(ppg_signal= ppg, sampling_rate=128)

#       # Find peaks
#       peaks = nk.ppg_findpeaks(ppg_cleaned, sampling_rate=128)

#       Feature_list = nk.hrv_nonlinear(peaks, sampling_rate=128, show=False)
#       print("Features ppg", Feature_list)



#     d_f1 = pd.DataFrame(Feature_list)
#     df=d_f1.to_numpy()
#     data=np.reshape(df,(1,-1))
#     df2=pd.concat([df2,pd.DataFrame(data=data)],ignore_index=True)

#     features=pd.concat([features, df2], ignore_index=True)

# features.shape

# Features= pd.concat([features, labels], axis=1, sort=False)
# Features = Features.dropna(how='any',axis=0)
# #Features
# #print("Features ppg", feature_list)
# #val=Features.iloc[:,:]
# #val.to_csv(r'feature_ppg.csv', index = False, header=True)
